In [0]:
%pip install yfinance

Downloading and Previewing yfinance data table for $NVDA

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

ticker = "NVDA"
end = datetime.today()
start = end - timedelta(days=365 * 10)  # ~10 years

data = yf.download(ticker, start=start, end=end)
data.head()


Create a new Unity Catalogue Volume to store the data

In [0]:
# Use Unity Catalog's default catalog "main"
spark.sql("CREATE CATALOG IF NOT EXISTS main")  # might already exist, that's fine
spark.sql("CREATE SCHEMA IF NOT EXISTS main.stocks")
spark.sql("USE CATALOG main")
spark.sql("USE SCHEMA stocks")

# Create a managed volume to hold the raw CSV files
spark.sql("CREATE VOLUME IF NOT EXISTS main.stocks.nvda_raw")


Store the yFinance Data in UC

In [0]:
raw_path = "/Volumes/main/stocks/nvda_raw"  # folder for Bronze source files
# If yfinance gave multi-index columns (e.g. ('Open', 'NVDA')), flatten them
if isinstance(data.columns, pd.MultiIndex):
    data.columns = [
        "_".join([str(c) for c in col if c != ""])
        for col in data.columns
    ]

# Reset index so we have a date column
data = data.reset_index()

cols_lower = {str(c).lower(): c for c in data.columns}

def get_col(possible_substrings, default=None):
    for key_lower, orig in cols_lower.items():
        for s in possible_substrings:
            if s in key_lower:
                return orig
    return default

date_col      = get_col(["date"])
open_col      = get_col(["open"])
high_col      = get_col(["high"])
low_col       = get_col(["low"])
close_col     = get_col(["close"])
adj_close_col = get_col(["adj close", "adj_close", "adjclose"])
volume_col    = get_col(["volume"])

# 4) Build a clean DataFrame with standard column names
data_clean = pd.DataFrame()
data_clean["date"]   = data[date_col]
data_clean["open"]   = data[open_col]
data_clean["high"]   = data[high_col]
data_clean["low"]    = data[low_col]
data_clean["close"]  = data[close_col]

# If there is no explicit adj close column, fall back to close
if adj_close_col is not None:
    data_clean["adj_close"] = data[adj_close_col]
else:
    data_clean["adj_close"] = data[close_col]

data_clean["volume"] = data[volume_col]

# 5) Write clean CSVs into the UC volume
raw_path = "/Volumes/main/stocks/nvda_raw"

spark_df = spark.createDataFrame(data_clean)

(
    spark_df
    .write
    .mode("overwrite")      # overwrite for now
    .option("header", True)
    .csv(raw_path)
)

In [0]:
display(
    spark.read.format("csv")
    .option("header", "true")
    .load(raw_path)
    .limit(5)
)
